In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
df = pd.read_csv('/content/drive/My Drive/cut.csv')
df.head()

,id,topic,heading,summary,date,sub-heading,content,tags
0,5ef0d321d83b3587cb500efc,"[""football""]","[""VAR not being used correctly: Barcelona coac...","[""Barcelona had a goal ruled out by VAR for of...","[""Updated: Jun 22, 2020 20:03 IST""]","[""Reuters""]","[""The Video Assistant Referee (VAR) system is ...","[""Quique Setien"",""Barcelona""]"
1,5ef0d324d83b3587cb500efd,"[""football""]","[""Mourinho hits back at Merson’s criticism of ...","[""Merson, a pundit with Sky Sports, said 26-ye...","[""Updated: Jun 22, 2020 20:30 IST""]","[""Reuters""]","[""Jose Mourinho launched an impassioned defenc...","[""Jose Mourinho"",""Harry Kane"",""Manchester unit..."
2,5efff14f86936ae10d2be3c8,"[""football""]","[""Morata double leads Atletico to 3-0 win over...","[""Morata earned a penalty when he was pulled d...","[""Updated: Jul 04, 2020 07:12 IST""]","[""Associated Press""]","[""Alvaro Morata scored twice to lead Atletico ...","[""Alvaro Morata"",""Mallorca"",""Diego Simeone"",""R..."
3,5efff10a86936ae10d2be3aa,"[""cricket""]","[""Sourav Ganguly made Sehwag open, brought in ...","[""Former India opener Wasim Jaffer, who made h...","[""Updated: Jul 04, 2020 08:03 IST""]","[""hindustantimes.com""]","[""Despite not being able to lead India to an I...","[""Sourav Ganguly"",""Wasim Jaffer"",""India"",""Zahe..."
4,5ef0d2e5d83b3587cb500edf,"[""cricket""]","[""Jasprit Bumrah is ‘a different beast’ when h...","[""The man in charge of the team currently is h...","[""Updated: Jun 22, 2020 20:57 IST""]","[""hindustantimes.com""]","[""Mumbai Indians are the most successful team ...","[""Mumbai Indians"",""Jasprit Bumrah"",""Mahela Jay..."


In [5]:
len(df.content.unique()),  len(df.content)

(18, 18)

In [6]:
df.content[0]

'["The Video Assistant Referee (VAR) system is not being used correctly in La Liga, Barcelona manager Quique Setien said on Monday following Real Madrid’s controversial win at Real Sociedad.","Madrid beat Sociedad 2-1 to climb above Barcelona at the top of La Liga but the home side were aggrieved because they had a goal ruled out by VAR for offside and Karim Benzema’s winner stood after he appeared to control the ball with his arm.","“There are things that we cannot control and that do not depend on us. Everybody saw what happened in Anoeta and everyone will draw their own conclusions,” Setien told reporters ahead of Tuesday’s match against Athletic Bilbao.","“It’s understandable that we think why there are some actions that are reviewed, the referee himself reviews them and they don’t do the same in other plays. They do it on some matches and in some others (they don’t). So you could really think that (VAR) is not being used correctly.”","Setien said controversies were part of footbal

In [ ]:
# time range for which these articles are published
#min(df.date_published).split("T")[0], max(df.date_published).split("T")[0]

In [7]:
# the number of articles in each section
Counter(df.topic)

Counter({'["cricket"]': 4, '["education"]': 11, '["football"]': 3})

In [8]:
df.isnull().sum()

id             0
topic          0
heading        0
summary        0
date           0
sub-heading    0
content        0
tags           2
dtype: int64

In [ ]:
#df.dropna(subset=['content'], inplace = True)
#df.dropna(subset = ['topic'], inplace = True)

In [ ]:
#df.shape

(148, 10)

In [9]:
import pandas as pd
import numpy as np
import re

import gensim
from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
# data preprocessing
stemmer = SnowballStemmer(language='english',ignore_stopwords=True)
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
# preview after preprocessing
doc_sample = df['content'][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['["The', 'Video', 'Assistant', 'Referee', '(VAR)', 'system', 'is', 'not', 'being', 'used', 'correctly', 'in', 'La', 'Liga,', 'Barcelona', 'manager', 'Quique', 'Setien', 'said', 'on', 'Monday', 'following', 'Real', 'Madrid’s', 'controversial', 'win', 'at', 'Real', 'Sociedad.","Madrid', 'beat', 'Sociedad', '2-1', 'to', 'climb', 'above', 'Barcelona', 'at', 'the', 'top', 'of', 'La', 'Liga', 'but', 'the', 'home', 'side', 'were', 'aggrieved', 'because', 'they', 'had', 'a', 'goal', 'ruled', 'out', 'by', 'VAR', 'for', 'offside', 'and', 'Karim', 'Benzema’s', 'winner', 'stood', 'after', 'he', 'appeared', 'to', 'control', 'the', 'ball', 'with', 'his', 'arm.","“There', 'are', 'things', 'that', 'we', 'cannot', 'control', 'and', 'that', 'do', 'not', 'depend', 'on', 'us.', 'Everybody', 'saw', 'what', 'happened', 'in', 'Anoeta', 'and', 'everyone', 'will', 'draw', 'their', 'own', 'conclusions,”', 'Setien', 'told', 'reporters', 'ahead', 'of', 'Tuesday’s', 'match', 'against', 'Athlet

In [12]:
# Preprocess the headline text, saving the results as ‘processed_docs’
df['processed_heading'] = df['heading'].map(preprocess)
df['processed_content'] = df['content'].map(preprocess)
df['processed_topic'] = df['topic'].map(preprocess)
df['processed_heading'] = df['processed_heading'].apply(lambda x: ' '.join(x))
df['processed_content'] = df['processed_content'].apply(lambda x: ' '.join(x))
df['processed_topic'] = df['processed_topic'].apply(lambda x: ' '.join(x))
df.head() 

,id,topic,heading,summary,date,sub-heading,content,tags,processed_heading,processed_content,processed_topic
0,5ef0d321d83b3587cb500efc,"[""football""]","[""VAR not being used correctly: Barcelona coac...","[""Barcelona had a goal ruled out by VAR for of...","[""Updated: Jun 22, 2020 20:03 IST""]","[""Reuters""]","[""The Video Assistant Referee (VAR) system is ...","[""Quique Setien"",""Barcelona""]",correct barcelona coach setien,video assist refere correct liga barcelona man...,footbal
1,5ef0d324d83b3587cb500efd,"[""football""]","[""Mourinho hits back at Merson’s criticism of ...","[""Merson, a pundit with Sky Sports, said 26-ye...","[""Updated: Jun 22, 2020 20:30 IST""]","[""Reuters""]","[""Jose Mourinho launched an impassioned defenc...","[""Jose Mourinho"",""Harry Kane"",""Manchester unit...",mourinho hit merson critic play style kane record,jose mourinho launch impass defenc harri kane ...,footbal
2,5efff14f86936ae10d2be3c8,"[""football""]","[""Morata double leads Atletico to 3-0 win over...","[""Morata earned a penalty when he was pulled d...","[""Updated: Jul 04, 2020 07:12 IST""]","[""Associated Press""]","[""Alvaro Morata scored twice to lead Atletico ...","[""Alvaro Morata"",""Mallorca"",""Diego Simeone"",""R...",morata doubl lead atletico mallorca liga,alvaro morata score twice lead atletico madrid...,footbal
3,5efff10a86936ae10d2be3aa,"[""cricket""]","[""Sourav Ganguly made Sehwag open, brought in ...","[""Former India opener Wasim Jaffer, who made h...","[""Updated: Jul 04, 2020 08:03 IST""]","[""hindustantimes.com""]","[""Despite not being able to lead India to an I...","[""Sourav Ganguly"",""Wasim Jaffer"",""India"",""Zahe...",sourav ganguli sehwag open bring yuvraj zaheer...,despit abl lead india trophi sole india joint ...,cricket
4,5ef0d2e5d83b3587cb500edf,"[""cricket""]","[""Jasprit Bumrah is ‘a different beast’ when h...","[""The man in charge of the team currently is h...","[""Updated: Jun 22, 2020 20:57 IST""]","[""hindustantimes.com""]","[""Mumbai Indians are the most successful team ...","[""Mumbai Indians"",""Jasprit Bumrah"",""Mahela Jay...",jasprit bumrah differ beast go middl mumbai in...,mumbai indian success team histori indian prem...,cricket


In [13]:
#Split the data into a "train" and "test". As well as select a metric in which to measure the similarity between articles. Think of the "train" set as the corpus.
# Think of the "test" set as the NYT articles that users are currently reading.

# move articles to an array
articles = df.processed_content.values

# move article section names to an array
sections = df.processed_topic.values

# move article web_urls to an array
headings = df.processed_heading.values

# shuffle these three arrays 
articles, sections, headings = shuffle(articles, sections, headings, random_state=4)

In [15]:
# split the shuffled articles into two arrays
n = 3

# one will have all but the last 10 articles -- think of this as your training set/corpus 
X_train = articles[:-n]
X_train_headings = headings[:-n]
X_train_sections = sections[:-n]

# the other will have those last 10 articles -- think of this as your test set/corpus 
X_test = articles[-n:]
X_test_headings = headings[-n:]
X_test_sections = sections[-n:]

In [16]:
# instantiate your vectorizor 
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [17]:
# fit the vectorizer 
tfidf_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [18]:
# transform both article splits 
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [19]:
#Builidng a Content Based Recommender
def get_top_n_rec_articles(X_train_tfidf, X_train, test_article, X_train_sections, X_train_headings, n = 5):
    '''This function calculates similarity scores bewteen a document and a corpus
    
       INPUT: vectorized document corpus, 2D array
              text document corpus, 1D array
              user article, 1D array
              article section names, 1D array
              article URLs, 1D array
              number of articles to recommend, int
              
       OUTPUT: top n recommendations, 1D array
               top n corresponding section names, 1D array
               top n corresponding URLs, 1D array
               similarity scores bewteen user article and entire corpus, 1D array
              '''
    # calculate similarity between the corpus (i.e. the "test" data) and the user's article
    similarity_scores = X_train_tfidf.dot(test_article.toarray().T)

    # get sorted similairty score indicies 
    sorted_indicies = np.argsort(similarity_scores, axis = 0)[::-1]

    # get sorted similarity socres
    sorted_sim_scores = similarity_scores[sorted_indicies]

    # get top n most similar documents
    top_n_recs = X_train[sorted_indicies[:n]]

    # get top n corresponding document section names
    rec_sections = X_train_sections[sorted_indicies[:n]]

    # get top n corresponding urls
    rec_headings = X_train_headings[sorted_indicies[:n]]
    
    # return recommendations and corresponding article meta-data
    return top_n_recs, rec_sections, rec_headings, sorted_sim_scores

In [20]:
# pick an article from the "test" set
# treat this as the article that the user is currently reading
k = 0
test_article = X_test_tfidf[k]

In [21]:
# return the top n most similar articles as recommendations 
top_n_recs, rec_sections, rec_headings, sorted_sim_scores = \
get_top_n_rec_articles(X_train_tfidf, X_train,  test_article,X_train_sections, X_train_headings, n = 5 )

In [22]:
#validate the results
#Compare the user's article and corresponding section name with the recommended articles and corresponding section names

# similarity scores
sorted_sim_scores[:5]

array([[[0.2246952 ]],

       [[0.22285593]],

       [[0.12201387]],

       [[0.09892844]],

       [[0.09773023]]])

In [23]:
# user's article
X_test[k]

'jose mourinho launch impass defenc harri kane form play style tottenham hotspur televis pundit paul merson suggest england striker struggl portugues manag merson pundit sport say year kane consid switch club mourinho continu adopt conserv style premier leagu like draw manchest unit week kane score seven goal game mourinho struggl impact unit match return hamstr surgeri januari mourinho say merson wrong judg base game record goal harri charg easi striker play mourinho tell report virtual news confer monday minut long respons mourinho go mention like didier drogba cristiano ronaldo karim benzema diego milito zlatan ibrahimov reel goal score stat tenur club dear paul lot respect mourinho add think harri kane problem score goal team especi fresh routin play good news injuri mourinho confirm forward luca moura avail recov injuri dele alli content serv match suspens time arriv attack player avail game mourinho say match final peopl play bench rotat player attack option match good'

In [24]:
# user's article's section name
X_test_sections[k]

'footbal'

In [25]:
# corresponding section names for top n recs 
rec_sections

array([['footbal'],
       ['footbal'],
       ['cricket'],
       ['cricket'],
       ['cricket']], dtype=object)

In [26]:
# top n article recs
top_n_recs

array([['alvaro morata score twice lead atletico madrid mallorca extend strong finish spanish leagu friday diego simeon team long titl race disput leader real madrid second place barcelona trail point atlético hottest team liga tick shift focus champion leagu final play portug month march leagu halt atlético struggl secur champion leagu berth season cours finish atlético remain undef seven match spanish leagu resum follow month stoppag covid pandem complic situat stoppag play play play great attitud morata say simeon domin releg threaten mallorca kickoff morata earn penalti pull aleksandar sedlar area goalkeep manuel reina block strike video review help refere order redo spot kick sedlar enter area earli morata step second rifl ball corner minut miss get tri footbal morata say penalti chelsea real madrid striker scoreboard half injuri time jorg koke resurrección steal ball play marco llorent clear reach line morata post home goal morata season leagu play mallorca muster comeback renan 

In [27]:
# corresonding URLs for top n recs 
rec_headings

array([['morata doubl lead atletico mallorca liga'],
       ['correct barcelona coach setien'],
       ['jasprit bumrah differ beast go middl mumbai indian head coach mahela jayawarden'],
       ['head coach phil simmon rejoin team mandatori quarantin covid test'],
       ['sourav ganguli sehwag open bring yuvraj zaheer harbhajan india open wasim jaffer name best captain play']],
      dtype=object)